In [1]:
import io
import time
import pickle
import eventlet
import requests
import pandas as pd
from tinydb import TinyDB
from stopwords_ca import batch


In [2]:
# Punctiation chars to be removed
punctuations = "!#$%&'()*+,-./:;?@[\]^_`{|}~"

# Added words to stopwords
batch += ["l", "d", "\\n"]


# Clean a text for NLP
# remove stopwords, punctuation, ...
def preprocess(text):

    if type(text) == float: text = "-"     # remove possible nan
    text = text.strip()                    # remove \n
    for p in punctuations:                 # remove puntuations
        try: text = text.replace(p, ' ')
        except: print(text)
        
     
    text = text.lower()                    # remove capital letters        
    text = ' '.join(                       # remove stopwords
        [x for x in text.split(' ') if x not in batch]
    )
    
    return text


# Preprocess a list of texts
def preprocess_text_list(text_list):
    return [(lambda x: preprocess(x))(x) for x in text_list]


def categoricPreprocess(column):

    var_list = list(set(column))
    cleanValues = preprocess_text_list(var_list)

    words = []
    for element in cleanValues: words += element.split(' ')
    uniqueWords = list(set(words))

    return(uniqueWords)


In [3]:
TIMEOUT_SECONDS = 10
numeric_description = []
categoric_description = []
errors = []

# db = TinyDB('data/catalog.json')
db = TinyDB('../data/catalog.json')


In [15]:
def rowDescription(content, numeric_description, categoric_description):
    
    # datasets with more columsn than limit will be ignored
    # needs to be done to avoid trying to read json files wrongly 
    # identified as csv, that will block the function
    LIMIT = 50
    
    try:
        # Convert to df and extract column names and description
        data = io.StringIO(content)
        df = pd.read_csv(data, sep=',')
        types = df.dtypes
        description = df.describe()
        columns = list(df.columns)

        if(len(columns) < LIMIT): # avoid incorreclty formated rows (json wrongly identified as csv)
            for column_name in columns:
                
                # column is numerical
                if(column_name in list(description.columns) 
                    and (types[column_name] == 'int64' or types[column_name] == 'float64')
                ):
                    desc = {
                        'datasetIndex': str(row['index']), 
                        'columnName': column_name, 
                        'description': description[column_name]
                    }
                    numeric_description.append(desc)
                    print(desc)
                
                # column is categorical
                else:
                    if(len(column_name) < 20): # avoid malformated columns (long colums names)
                        desc = {
                            'datasetIndex': str(row['index']), 
                            'columnName': column_name, 
                            'description': categoricPreprocess(df[column_name])
                        }

                        categoric_description.append(desc)
            print("Success loading dataset")
        else:
            print("Error in format, avoiding load")
    except:
        raise()
        print("Error loading dataset")

In [20]:
for dataset in db.all():
    print("Index: ", dataset['index'])
    
    # Try do download file from url
    url = dataset["download_url"]
    try:
        with eventlet.Timeout(TIMEOUT_SECONDS):
            req = requests.get(url)
    except:
        print("timeout, too slow")
        errors.append({"index": row['index'], "error":'timeout'})
        
    
    #### Try decoding the file ####
    try:
        # content can be downloaded in utf-8
        content = req.content.decode("utf-8") 

        # Check if its json
        if((content[0] == '[' and content[len(content)-1] == ']')
            or content[0] == '{' and content[len(content)-1] == '}'
        ):
            print("Format identified: json")
            # Decode json and convert to csv
            # ....

        else:
            # assume utf-8, try to get row description
            rowDescription(content, numeric_description, categoric_description)

            try:
                print("is csv")
                rowDescription(content, numeric_description, categoric_description)
                print("Format identified: csv utf-8")
            except:
                print('ERROR (1): no successful rowDescription')

    except:
        # content cant be downloaded in utf-8, try utf-16
        try:
            content = req.content.decode("utf-16")
        except:
            print("ERROR: no format identified")
        else:
            print('Format identified: csv utf-16')
            try:
                rowDescription(content, numeric_description, categoric_description)
            except:
                print('ERROR (2): no successful rowDescription')   
                
    print('---------------')


Index:  0
ERROR: no format identified
---------------
Index:  1
timeout, too slow


NameError: name 'row' is not defined

In [16]:
for dataset in db.all():
    print("Index: ", dataset['index'])
    
    # Try do download file from url
    url = dataset["download_url"]
    try:
        with eventlet.Timeout(TIMEOUT_SECONDS):
            req = requests.get(url)
    except:
        print("timeout, too slow")
        errors.append({"index": row['index'], "error":'timeout'})
        
    #### Try decoding the file ####
    try:
        # content can be downloaded in utf-8
        content = req.content.decode("utf-8")
        
        # Check if its json
        if((content[0] == '[' and content[len(content)-1] == ']')
            or content[0] == '{' and content[len(content)-1] == '}'
        ):
            print("Format identified: json")
            # Decode json and convert to csv
            # ....

        else:
            # assume utf-8, try to get row description
            print("is csv")
            rowDescription(content, dataset["download_url"], numeric_description, categoric_description)
    
    except:
        # content cant be downloaded in utf-8, try utf-16
        pass
    break

Index:  0
is csv


In [52]:
def rowDescriptionTest(content, datasetIndex, numeric_description, categoric_description):
    LIMIT = 50
    data = io.StringIO(content)
    df = pd.read_csv(data, sep=',')
    types = df.dtypes
    description = df.describe()
    columns = list(df.columns)
        
    
    if(len(columns) < LIMIT): # avoid incorreclty formated rows (json wrongly identified as csv)
        for column_name in columns:

            # column is numerical
            if(column_name in list(description.columns) 
                and (types[column_name] == 'int64' or types[column_name] == 'float64')
            ):
                print(column_name, "numerical")
                desc = {
                        'datasetIndex': str(datasetIndex), 
                        'columnName': column_name, 
                        'description': list(description[column_name])
                }
                numeric_description.append(desc)
                print(desc)

            # column is categorical
            else:
                print(column_name, "categorical")
                if(len(column_name) < 20): # avoid malformated columns (long colums names)
                    desc = {
                        'datasetIndex': str(datasetIndex), 
                        'columnName': column_name, 
                        'description': categoricPreprocess(df[column_name])
                    }

                    categoric_description.append(desc)
                    print(desc)


        print("Success loading dataset")
    else:
        print("Error in format, avoiding load")


def main():

    numeric_description = []
    categoric_description = []

    for dataset in db.all()[0:10]:
        print("Index: ", dataset['index'])
        url = dataset["download_url"]
        print(url)

        # extract content from url   
        try:
            with eventlet.Timeout(TIMEOUT_SECONDS):
                req = requests.get(url)
        except:
            print("ERROR: timeout, too slow")
            errors.append({"index": dataset['index'], "error":'timeout'})

        # process
        try:
            # Identified as utf-8
            content = req.content.decode("utf-8")
            # Check if its json
            if((content[0] == '[' and content[len(content)-1] == ']')
                or content[0] == '{' and content[len(content)-1] == '}'
            ):
                print("Format identified: json")
                # Decode json and convert to csv
                # ....
            else:   
                print("Format identified: csv utf-8")
                rowDescriptionTest(content, dataset["index"], numeric_description, categoric_description)
        except: 
            # content cant be downloaded in utf-8, try utf-16
            print('Format not identified')
            pass
        print('-------------------')

main()


Index:  0
https://opendata-ajuntament.barcelona.cat/data/dataset/808daafa-d9ce-48c0-925a-fa5afdb1ed41/resource/4cc59b76-a977-40ac-8748-61217c8ff367/download
Format identified: csv utf-8
CODI_DISTRICTE numerical
{'datasetIndex': '0', 'columnName': 'CODI_DISTRICTE', 'description': [73.0, 6.2465753424657535, 2.807632563001791, 1.0, 4.0, 7.0, 8.0, 10.0]}
NOM_DISTRICTE categorical
{'datasetIndex': '0', 'columnName': 'NOM_DISTRICTE', 'description': ['eixample', 'ciutat', 'horta', 'andreu', 'corts', 'gràcia', 'vella', 'barris', 'martí', 'sarrià', 'nou', 'gervasi', 'guinardó', 'montjuïc', 'sant', 'sants']}
CODI_BARRI numerical
{'datasetIndex': '0', 'columnName': 'CODI_BARRI', 'description': [73.0, 37.0, 21.217131442932306, 1.0, 19.0, 37.0, 55.0, 73.0]}
NOM_BARRI categorical
{'datasetIndex': '0', 'columnName': 'NOM_BARRI', 'description': ['', 'congrés', 'vallbona', 'guatlla', 'bon', 'camp', 'tres', 'salut', 'penitents', 'olímpica', 'dreta', 'pienc', 'família', 'barri', 'vall', 'galvany', 'clot'

Format identified: csv utf-8
Numero_expedient categorical
{'datasetIndex': '3', 'columnName': 'Numero_expedient', 'description': ['2021s002836', '2021s001210', '2021s005913', '2021s004546', '2021s001699', '2021s005779', '2021s001454', '2021s002223', '2021s003409', '2021s000543', '2021s003351', '2021s000151', '2021s000111', '2021s004597', '2021s006746', '2021s003878', '2021s005949', '2021s005379', '2021s004473', '2021s000532', '2021s007300', '2021s007020', '2021s005625', '2021s000033', '2021s000432', '2021s007011', '2021s002940', '2021s007327', '2021s007075', '2021s004446', '2021s000722', '2021s003391', '2021s000006', '2021s006245', '2021s003968', '2021s003091', '2021s004420', '2021s004010', '2021s001521', '2021s003765', '2021s001049', '2021s001612', '2021s006880', '2021s006139', '2021s005009', '2021s000790', '2021s002543', '2021s007188', '2021s006988', '2021s003198', '2021s001650', '2021s004108', '2021s004679', '2021s007497', '2021s002230', '2021s006825', '2021s001982', '2021s005807', 

Format identified: csv utf-8
Numero_Expedient categorical
{'datasetIndex': '4', 'columnName': 'Numero_Expedient', 'description': ['2021s002836', '2021s001210', '2021s005913', '2021s004546', '2021s001699', '2021s005779', '2021s001454', '2021s002223', '2021s003409', '2021s000543', '2021s000151', '2021s000111', '2021s004597', '2021s006746', '2021s003878', '2021s005949', '2021s005379', '2021s004473', '2021s000532', '2021s007300', '2021s007020', '2021s005625', '2021s000033', '2021s000432', '2021s002940', '2021s007075', '2021s004446', '2021s000722', '2021s003391', '2021s000006', '2021s003968', '2021s003091', '2021s004420', '2021s004010', '2021s001521', '2021s003765', '2021s001049', '2021s001612', '2021s006880', '2021s006139', '2021s005009', '2021s000790', '2021s002543', '2021s007188', '2021s006988', '2021s001650', '2021s004108', '2021s004679', '2021s007497', '2021s002230', '2021s006825', '2021s001982', '2021s005807', '2021s001966', '2021s002198', '2021s002876', '2021s004680', '2021s000504', 

Format identified: csv utf-8
Numero_expedient categorical
{'datasetIndex': '5', 'columnName': 'Numero_expedient', 'description': ['2021s002836', '2021s001210', '2021s005913', '2021s004546', '2021s001699', '2021s005779', '2021s001454', '2021s002223', '2021s003409', '2021s000543', '2021s003351', '2021s000151', '2021s000111', '2021s004597', '2021s006746', '2021s003878', '2021s005949', '2021s005379', '2021s004473', '2021s000532', '2021s007300', '2021s007020', '2021s005625', '2021s000033', '2021s000432', '2021s007011', '2021s002940', '2021s007327', '2021s007075', '2021s004446', '2021s000722', '2021s003391', '2021s000006', '2021s006245', '2021s003968', '2021s003091', '2021s004420', '2021s004010', '2021s001521', '2021s003765', '2021s001049', '2021s001612', '2021s006880', '2021s006139', '2021s005009', '2021s000790', '2021s002543', '2021s007188', '2021s006988', '2021s003198', '2021s001650', '2021s004108', '2021s004679', '2021s007497', '2021s002230', '2021s006825', '2021s001982', '2021s005807', 

Format identified: csv utf-8
Numero_expedient categorical
{'datasetIndex': '6', 'columnName': 'Numero_expedient', 'description': ['', '2021s002836', '2021s001210', '2021s005913', '2021s004546', '2021s001699', '2021s005779', '2021s001454', '2021s002223', '2021s003409', '2021s000543', '2021s003351', '2021s000151', '2021s000111', '2021s004597', '2021s006746', '2021s003878', '2021s005949', '2021s005379', '2021s004473', '2021s000532', '2021s007300', '2021s007020', '2021s005625', '2021s000033', '2021s000432', '2021s007011', '2021s002940', '2021s007327', '2021s007075', '2021s004446', '2021s000722', '2021s003391', '2021s000006', '2021s006245', '2021s003968', '2021s003091', '2021s004420', '2021s004010', '2021s001521', '2021s003765', '2021s001049', '2021s001612', '2021s006880', '2021s006139', '2021s005009', '2021s000790', '2021s002543', '2021s007188', '2021s006988', '2021s003198', '2021s001650', '2021s004108', '2021s004679', '2021s007497', '2021s002230', '2021s006825', '2021s001982', '2021s00580

Format identified: csv utf-8
Catàleg categorical
{'datasetIndex': '7', 'columnName': 'Catàleg', 'description': ['', 'justícia', '2022', 'dac', 'imssb', 'transversal', 'global', 'imi', 'lgtbi', 'ds']}
Àmbit d’actuació    categorical
{'datasetIndex': '7', 'columnName': 'Àmbit d’actuació   ', 'description': ['', 'informació', 'transversalitat', 'projectes', 'gestió', 'llengua', 'normativa', 'salut', 'econòmic', 'financier', 'habilitats', 'gènere', 'administració', 'desenvolupament', 'transversal', 'comandament', 'actualització', 'electrònica', 'descentralitzada', 'laborals', 'comunicació', 'competencial', 'col·laborativa', 'seguretat', 'igualtat', 'capacitats', 'tecnologies']}
Nom acció formativa categorical
{'datasetIndex': '7', 'columnName': 'Nom acció formativa', 'description': ['', 'aprenent', 'presentacions', 'projectes', 'virtuals', 'gestió', 'pràctiques', 'enfocament', '"mindfulness"', 'formularis', 'interna', 'administratius', 'teòrico', 'canvi', 'joncs', 'adolescents', 'intel·lec

Format identified: csv utf-8
Concepte categorical
{'datasetIndex': '8', 'columnName': 'Concepte', 'description': ['gossos', 'rescatats', 'gats', 'adoptats', 'recollits']}
Gener numerical
{'datasetIndex': '8', 'columnName': 'Gener', 'description': [6.0, 30.666666666666668, 14.569374271624252, 10.0, 20.0, 35.0, 41.0, 46.0]}
Febrer numerical
{'datasetIndex': '8', 'columnName': 'Febrer', 'description': [6.0, 26.0, 13.053735097664577, 8.0, 20.0, 25.5, 30.25, 47.0]}
Març numerical
{'datasetIndex': '8', 'columnName': 'Març', 'description': [0.0, nan, nan, nan, nan, nan, nan, nan]}
Abril numerical
{'datasetIndex': '8', 'columnName': 'Abril', 'description': [0.0, nan, nan, nan, nan, nan, nan, nan]}
Maig numerical
{'datasetIndex': '8', 'columnName': 'Maig', 'description': [0.0, nan, nan, nan, nan, nan, nan, nan]}
Juny numerical
{'datasetIndex': '8', 'columnName': 'Juny', 'description': [0.0, nan, nan, nan, nan, nan, nan, nan]}
Juliol numerical
{'datasetIndex': '8', 'columnName': 'Juliol', 'descr

In [51]:
numeric_description

[{'datasetIndex': '0',
  'columnName': 'CODI_DISTRICTE',
  'description': [73.0,
   6.2465753424657535,
   2.807632563001791,
   1.0,
   4.0,
   7.0,
   8.0,
   10.0]},
 {'datasetIndex': '0',
  'columnName': 'CODI_BARRI',
  'description': [73.0,
   37.0,
   21.217131442932306,
   1.0,
   19.0,
   37.0,
   55.0,
   73.0]},
 {'datasetIndex': '1',
  'columnName': 'CODI_DISTRICTE',
  'description': [73.0,
   6.2465753424657535,
   2.807632563001791,
   1.0,
   4.0,
   7.0,
   8.0,
   10.0]},
 {'datasetIndex': '1',
  'columnName': 'CODI_BARRI',
  'description': [73.0,
   37.0,
   21.217131442932306,
   1.0,
   19.0,
   37.0,
   55.0,
   73.0]},
 {'datasetIndex': '2',
  'columnName': 'Codi_districte',
  'description': [7653.0,
   4.951783614268914,
   3.016490834865949,
   -1.0,
   2.0,
   4.0,
   8.0,
   10.0]},
 {'datasetIndex': '2',
  'columnName': 'Codi_barri',
  'description': [7653.0,
   27.849340128054358,
   22.16013952434831,
   -1.0,
   8.0,
   21.0,
   44.0,
   73.0]},
 {'datasetI

--------------------------------------------------------

In [57]:
import json

with open('../data/categorical_description.json', 'r') as f:
    categorical_description = json.load(f)

In [59]:
categorical_description[0]

{'datasetIndex': '0',
 'columnName': 'NOM_DISTRICTE',
 'description': ['sant',
  'barris',
  'montjuïc',
  'martí',
  'nou',
  'horta',
  'eixample',
  'ciutat',
  'sarrià',
  'vella',
  'gràcia',
  'gervasi',
  'guinardó',
  'corts',
  'sants',
  'andreu']}

In [62]:
columnNames = []
texts = []

for column in categorical_description:
    columnName = column['columnName']
    values = column['description']
    joinValues = ' '.join(values)
    
    texts.append(joinValues)
    columnNames.append(columnName)

    break